In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sequence_generator import sequence_generation
__file__ = "preprocess_data_generation.ipynb"
dirname = os.path.dirname(os.path.abspath(os.path.dirname(__file__)))
sys.path.append(dirname)

In [2]:
df = pd.read_csv(dirname + "/Data/csv_folder/매출TR_202001.csv")
df.drop(columns=["Unnamed: 0"], inplace = True)
df.dropna(subset= ["회원카드번호"], inplace = True)
df.drop_duplicates(inplace = True)
df.reset_index(inplace = True, drop = True)
loc_df = pd.read_csv(dirname + "/Data/base_mapping_file.csv")
custom_inf = ["성별", "회원구분", "회원등급", "업종유형"]

In [8]:
id_list = []
for i in tqdm(range(len(df))):
    id_list.append((df.거래번호[i], int(df.회원카드번호[i])))
id_list = list(set(id_list))

100%|██████████| 433966/433966 [00:10<00:00, 41290.28it/s]


In [9]:
#Stream data 그자체 생성하는 code (고객정보 + 상품 선택 순서정보) -> json file
dic  = {}
start_point = [24,0]
end_point = [12,0]
for idx, id in tqdm(id_list):
    temp = df[(df.거래번호 == idx) & (df.회원카드번호 == id)]
    temp.reset_index(inplace= True, drop = True)
    path_list = []
    for i in range(len(temp)):
        good_loc = loc_df[(loc_df.대분류 == temp.loc[i, "대분류"]) &(loc_df.중분류 == temp.loc[i, "중분류"])& (loc_df.소분류 == temp.loc[i, "소분류"])][["row","column"]].values
        if len(good_loc) == 0:
            pass
        else:
            path_list.append(list(good_loc[0]))

    if len(temp) == len(path_list):
        if temp.성별[0] in ["M", "F"]:
            if temp.회원구분[0] != "직원":
                if str(temp.회원등급[0]) != "nan":
                    if str(temp.업종유형[0]) != "nan":
                        if (idx,id) not in dic.keys():
                            dic[str((idx,id))] = {
                                "purchase_list" : [], 
                                "성별" : temp.성별[0], 
                                "회원구분" : temp.회원구분[0], 
                                "회원등급" : temp.회원등급[0], 
                                "업종유형" :temp.업종유형[0] 
                                }
                        
                        result_list = sequence_generation(start_point, path_list, end_point)
                        result_list.remove(len(result_list)-1)
                        result_list.remove(0)
                        result_list = list(np.array(result_list)-1)
                        
                        for i in result_list:
                            dic[str((idx,id))]["purchase_list"].append(
                                {
                                    "상품명" : temp.loc[i,"상품명"],
                                    "상품코드" : str(temp.loc[i,"상품코드"]),
                                    "대분류" : temp.loc[i,"대분류"],
                                    "중분류" : temp.loc[i,"중분류"],
                                    "소분류" : temp.loc[i,"소분류"],
                                    "row" : str(path_list[i][0]),
                                    "column" : str(path_list[i][1])
                                }
                            )

100%|██████████| 62783/62783 [15:49<00:00, 66.15it/s]


In [10]:
import json
with open(dirname+"/Data/json_file/purchase_202001_ver4.json", 'w') as jsonfile:
    json.dump(dic, jsonfile)


In [6]:
price_df = pd.DataFrame()
price_df["상품명"] = df["상품명"]
price_df["단가"] = df["단가"]


In [ ]:
price_df.drop_duplicates(inplace = True)

In [ ]:
price_df